# Notebook 2: Solving the underlying transmission process

In this notebook we look at the underlying transmission process described in the main paper, and how this is solved in Julia. The underlying ODE is represented and solved using the DifferentialEquations.jl package, and familiarity with that package would be useful at this point.

The underlying transmission dynamics in each county was represented as an ODE:
$$
    \dot{S}(t) = -{S(t)\over P_{eff} N} \gamma_t R_t I(t),\\ 
    \dot{E}(t) = {S(t)\over P_{eff} N} \gamma_t R_t I(t) - \sigma E(t),\\
    \dot{I}(t) = \sigma E(t) - \gamma I(t),\\
    \dot{R}(t) = \gamma I(t),\\
    \dot{C}(t) = {S(t)\over P_{eff} N} \gamma_t R_t I(t).
$$
Where $1/\sigma = 3.1$ days was the mean time between infection and infectiousness, and $1/\gamma = 2.4$ days was chosen as the mean infectious period to match a 5.5 day serial interval. The system is a standard SEIR model with one exception: an *effective* population size of less than $N$ due to the effect of heterogeneity in the underlying population (also $S(0) = P_{eff} N - E(0) - I(0)$). The dynamic variable $C(t)$ tracks cumulative infections. $R_t = R_0 $